# CNN (Convolutional Neural Network)

In [5]:
from IPython.display import Image

## 텐서플로우로 CNN 실습하기
텐서플로우로 아래 그림과 동일한 CNN을 직접 구현하고, MNIST 손글씨로 학습 및 테스트를 진행보도록 하겠습니다.

In [6]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/practice_cnn.png", width=800, height=200)

In [7]:
import tensorflow as tf

### 데이터 획득

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

### 학습데이터에서 검증 데이터 분리하기
학습 중간마다 검증 데이터로 모델의 성능을 측정하면 다음과 같은 장점이 있습니다.  
1) 모델 학습이 제대로 진행되는 지 검증 정확도로 확인할 수 있습니다.  
2) 학습 정확도는 올라가는 데 검증 정확도가 안올라가거나 떨어질 시, 조기 종료를 구현할 수 있습니다.

In [9]:
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]

In [10]:
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

train data has 50000 samples
every train data is 28 * 28 image


In [11]:
print("validation data has " + str(x_val.shape[0]) + " samples")
print("every train data is " + str(x_val.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

validation data has 10000 samples
every train data is 28 * 28 image


**0** 부터 **255** 까지의 그레이 스케일을 확인할 수 있습니다.

In [12]:
# sample to show gray scale values
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


**0** 부터 **9**까지의 이미지에 해당하는 숫자를 확인할 수 있습니다.

In [13]:
# sample to show labels for first train data to 10th train data
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


테스트 데이터는 **10000** 개의 샘플을 가지고 있습니다.  
모든 테스트 데이터는 **28 * 28** 의 이미지입니다.

In [14]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


### 데이터 구조 변경하기
입력 레이어에 데이터를 넣기 위해서 데이터의 구조를 변경해줍니다.

In [15]:
import numpy as np
x_train = np.reshape(x_train, (50000,28,28,1))
x_val = np.reshape(x_val, (10000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(50000, 28, 28, 1)
(10000, 28, 28, 1)


### 데이터 정규화
데이터 정규화는 보통 학습 시간을 단축하고, 더 나은 성능을 구하도록 도와줍니다.  
MNIST 데이터의 모든 값은 0부터 255의 범위 안에 있으므로, 255로 값을 나눠줌으로써, 모든 값을 0부터 1 사이의 값으로 정규화합니다.  

In [16]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

### 실제값을 one hot encoding으로 변경하기
손실 함수에서 크로스 엔트로피를 계산하기 위해, 실제값은 one hot encoding 값으로 변경합니다.

In [17]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

### CNN 텐서플로우로 구현하기

In [18]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/practice_cnn.png", width=800, height=200)

MNIST 데이터 형태 그대로 28 X 28의 포맷을 그대로 입력 데이터로 사용합니다.  
타겟은 0부터 9까지의 숫자입니다.

In [19]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

다음으로 파라미터(weight, bias) 초기값을 설정해줍니다.

In [20]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

padding='SAME'은 레이어에 입력된 피쳐맵의 사이즈와 동일하게 피쳐맵을 출력하도록 설정해줍니다.
예를 들어, CONV1에 28 X 28의 이미지 데이터가 입력되면, 동일한 크기인 28 X 28 형태의 피쳐맵이 출력되게 됩니다.

In [21]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

첫번째 CONV 레이어는 총 16개의 필터를 가지고 있고 필터의 사이즈는 5 X 5입니다.
편향값(bias)는 필터의 갯수만큼 만들어줍니다.

In [22]:
W_conv1 = weight_variable([5, 5, 1, 16])
b_conv1 = bias_variable([16])

활성화함수로 relu를 사용합니다.

In [23]:
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)

CONV 레이어 다음으로 풀링 레이어를 적용하여 activation map의 크기를 줄여줍니다. activation map의 크기를 줄여줌으로써, 파라미터가 줄어들어 모델 크기가 작아지고, 과대적합의 위험도 감소시켜줍니다.

In [24]:
h_pool1 = max_pool_2x2(h_conv1)

풀링레이어의 영향으로, activation map의 사이즈는 14 X 14가 되었습니다. 이 값은 다음에 이어지는 CONV2의 입력으로 들어가게 됩니다. CONV2는 총 32개의 필터를 가지고 있습니다.

In [25]:
W_conv2 = weight_variable([5, 5, 16, 32])
b_conv2 = bias_variable([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

풀링레이어 이후의 activation map의 사이즈는 7 X 7이 됩니다.  

#### FC (Fully Connected Layer)
FC는 CONV를 통해 추출된 이미지의 특징들을 입력으로 받아 0부터 9까지의 숫자 중 하나로 이미지를 분류합니다.
아래 코드에서 tf.reshape를 사용하여 모든 특징들을 하나의 배열로 형변환합니다. 형변환된 특징들은 FC의 입력값이 됩니다.

In [26]:
W_fc1 = weight_variable([7 * 7 * 32, 128])
b_fc1 = bias_variable([128])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

FC 영역에는 FC1, FC2 총 2개의 히든 레이어가 존재합니다. FC1은 128개의 노드, FC2에는 10개의 노드가 존재합니다. FC2에 10개의 노드가 존재하는 이유는 FC2의 10개 노드의 값들을 소프트맥스에 입력시켜서 각 노드별 확률을 구하기 위해서입니다. 각 노드는 숫자 0부터 9를 의미하며, 이 예측값은 크로스 엔트로피를 통해 실제값과의 차이를 계산하는 데 사용됩니다.

In [27]:
W_fc2 = weight_variable([128, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

아래와 같이 크로스엔트로피를 설정해줍니다.

In [28]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y_, logits=y_conv))

아담 옵티마이저를 사용해서 모델을 최적화합니다. 학습률은 0.001로 설정하였습니다.

In [29]:
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

아래 코드는 정확도를 구하기 위해서 사용합니다.

In [30]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### 학습 및 테스트
이제 CNN 모델을 MNIST 데이터로 학습해보도록 하겠습니다. 단 3번의 주기만으로도 놀라온 정확도가 확인됩니다. 이번 실습에서는 미니배치를 사용하여 모델을 최적화합니다. 미니배치는 총 500개의 학습데이터가 들어있고, 미니배치마다 파라미터를 조정해줍니다.

In [31]:
# initialize
init = tf.global_variables_initializer()

# train hyperparameters
epoch_cnt = 3
batch_size = 500
iteration = len(x_train) // batch_size

# Start training
with tf.Session() as sess:
    tf.set_random_seed(777)
    # Run the initializer
    sess.run(init)
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            if i%10 == 0:
                train_acc = accuracy.eval(
                        feed_dict={x:x_train[start: end], 
                                    y_: y_train[start: end]})
                print("step "+str(i)+ ": train acc: "+str(train_acc))
            train_step.run(
                feed_dict={
                    x:x_train[start: end], y_: y_train[start: end]})
            start += batch_size; end += batch_size    
        
        # Validate model
        val_accuracy = accuracy.eval(feed_dict={x:x_val, y_: y_val})
        print("validation accuracy: "+str(val_accuracy))
        
    test_accuracy = accuracy.eval(feed_dict={x:x_test, y_: y_test}) 
    print("test accuracy: "+str(test_accuracy))

step 0: train acc: 0.11
step 10: train acc: 0.646
step 20: train acc: 0.83
step 30: train acc: 0.866
step 40: train acc: 0.89
step 50: train acc: 0.914
step 60: train acc: 0.92
step 70: train acc: 0.942
step 80: train acc: 0.94
step 90: train acc: 0.926
validation accuracy: 0.9535
step 0: train acc: 0.948
step 10: train acc: 0.952
step 20: train acc: 0.946
step 30: train acc: 0.958
step 40: train acc: 0.958
step 50: train acc: 0.972
step 60: train acc: 0.968
step 70: train acc: 0.968
step 80: train acc: 0.972
step 90: train acc: 0.97
validation accuracy: 0.9717
step 0: train acc: 0.968
step 10: train acc: 0.982
step 20: train acc: 0.96
step 30: train acc: 0.966
step 40: train acc: 0.982
step 50: train acc: 0.976
step 60: train acc: 0.978
step 70: train acc: 0.97
step 80: train acc: 0.978
step 90: train acc: 0.97
validation accuracy: 0.9787
test accuracy: 0.9778


단 **3** 번의 주기만으로도, **97.7%** 의 정확도를 확인할 수 있습니다.